# Exercises

## 1. What is the approximate depth of a decision tree trained (without restrictions) on a training set with one million instances?

The approximate depth of a decision tree trained (without restrictions) on a training set with one million instances is about $log_2{10^6} \approx 20$

## 2. Is a node’s Gini impurity generally lower or higher than its parent’s? Is it generally lower/higher, or always lower/higher?

A node's Gini impurity is generally lower than its parents.

## 3. If a decision tree is overfitting the training set, is it a good idea to try decreasing max_depth?

Definitely yes, because `max_depth` parameter is a regularization, will constrains the model.

## 4. If a decision tree is underfitting the training set, is it a good idea to try scaling the input features?

No, because the decision tree doesn't need the cleaning data. Scaling the input will not change the result.

## 5. If it takes one hour to train a decision tree on a training set containing one million instances, roughly how much time will it take to train another decision tree on a training set containing ten million instances? Hint: consider the CARTalgorithm’s computational complexity.

Computational Complexity: $O(n \times m log_2 (m))$ If you train ten million instances equal 10 times of initial the number of instances. We will set a scale: $(n \times 10m log2_(10m)) / (n \times m log2(m)) = 10 \times log2(10m) / log2(m)$ Then subs $m = 10^6 \to 11.7 \to$ take 11.7 hours to train.

## 6. If it takes one hour to train a decision tree on a given training set, roughly how much time will it take if you double the number of features?

Computational Complexity: $O(n \times m log2(m))$ We will set a scale again:

$(n \times m log2(m)) / (2n \times m log2(m)) = 1/2 \to $ take $2$ hour to train. $\to$ take double time to train.

## 7. Train and fine-tune a decision tree for the moons dataset by following these steps:

a. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

b. Use train_test_split() to split the dataset into a training set and a test set.

c. Use grid search with cross-validation (with the help of the GridSearchCV
class) to find good hyperparameter values for a DecisionTreeClassifier.
Hint: try various values for max_leaf_nodes.

d. Train it on the full training set using these hyperparameters, and measure
your model’s performance on the test set. You should get roughly 85% to 87%
accuracy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# generate data
X, y = make_moons(n_samples=10000, noise = 0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
param = {
    "max_leaf_nodes": [2, 3, 4, 5, 6, 7, 8, 9, 10]
}

my_tree = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(my_tree, param, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'max_leaf_nodes': 4}


In [2]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.863


## 8. Grow a forest by following these steps:

a. Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this.

b. Train one decision tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 decision trees on the test
set. Since they were trained on smaller sets, these decision trees will likely
perform worse than the first decision tree, achieving only about 80% accuracy.

c. Now comes the magic. For each test set instance, generate the predictions of
the 1,000 decision trees, and keep only the most frequent prediction (you can
use SciPy’s mode() function for this). This approach gives you majority-vote
predictions over the test set.

d. Evaluate these predictions on the test set: you should obtain a slightly higher
accuracy than your first model (about 0.5 to 1.5% higher). Congratulations,
you have trained a random forest classifier!

In [3]:
from sklearn.base import clone
forest = [clone(grid_search.best_estimator_) for i in range(1000)]
# a.
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=1000, test_size=len(X_train) - 100, random_state=42)
mini_sets = []
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))
# b.

accuracy_scores = []
for i in range(len(mini_sets)):
    forest[i].fit(mini_sets[i][0], mini_sets[i][1])
    y_pred = forest[i].predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

print(np.mean(accuracy_scores))


0.8347645


In [4]:
# c.
Y_pred = []

for tree in forest:
    Y_pred.append(tree.predict(X_test))

from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [5]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8715